## Analysis of Data and Classification Models in Letter Recognition
### by Joshua Gabella
_______________________________________________________________________________________________________________________________


Imports

In [73]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [63]:
df = pd.read_csv("letter-recognition.data", index_col=False)
data = df.to_numpy()
HKdata = data[np.logical_or(data[:,0]=='H', data[:,0] == 'K'), :]
MYdata = data[np.logical_or(data[:,0]=='M', data[:,0] == 'Y'), :]
DOdata = data[np.logical_or(data[:,0]=='D', data[:,0] == 'O'), :]

## H and K Analysis

In [64]:
training_data, test_data = train_test_split(HKdata, test_size=0.1, random_state=0)
training_y = training_data[:, 0]
training_X = training_data[:, 1:]
test_y = test_data[:, 0]
test_X = test_data[:, 1:]

### Kernel SVM - H/K

In [65]:
cs = [0.1, 1, 3, 5, 10]
scores = {param_value : cross_val_score(svm.SVC(kernel='rbf', C = param_value), training_X, training_y, cv=5) for param_value in cs}
means = {param_value : np.mean(scores[param_value]) for param_value in (list)(scores.keys())}
stdevs = {param_value : np.std(scores[param_value]) for param_value in (list)(scores.keys())}
for i in scores:
    print(f"C value: {i} --> Mean Score: {np.mean(scores[i])}, Standard Deviation: {np.std(scores[i])}")

C value: 0.1 --> Mean Score: 0.9222641509433963, Standard Deviation: 0.012309061456830254
C value: 1 --> Mean Score: 0.9607547169811321, Standard Deviation: 0.010566037735849054
C value: 3 --> Mean Score: 0.969811320754717, Standard Deviation: 0.007915538476755858
C value: 5 --> Mean Score: 0.9743396226415093, Standard Deviation: 0.007317252614968042
C value: 10 --> Mean Score: 0.9743396226415093, Standard Deviation: 0.005546014511961911


In [66]:
clf = GridSearchCV(svm.SVC(), {
    'C':[0.1, 1, 5, 10, 20],
    'kernel':['rbf', 'linear', 'sigmoid'],
    'gamma':['auto', 'scale']
}, cv=5)
clf.fit(training_X, training_y)
param_SVM_results = pd.DataFrame(clf.cv_results_)
trimmed_SVM_results = param_SVM_results[['param_C', 'param_kernel', 'param_gamma', 'mean_test_score', 'std_test_score']]



In [67]:
trimmed_SVM_results.sort_values('mean_test_score', ascending=False)

,param_C,param_kernel,param_gamma,mean_test_score,std_test_score
12,5,rbf,auto,0.981887,0.003697
18,10,rbf,auto,0.981132,0.002387
24,20,rbf,auto,0.981132,0.002387
27,20,rbf,scale,0.978113,0.005006
6,1,rbf,auto,0.975849,0.003848
21,10,rbf,scale,0.974340,0.005546
15,5,rbf,scale,0.974340,0.007317
9,1,rbf,scale,0.960755,0.010566
25,20,linear,auto,0.928302,0.011194
19,10,linear,auto,0.928302,0.011194


---
### Decision Tree Classifier - H/K

In [68]:
clf = GridSearchCV(tree.DecisionTreeClassifier(), {
    'criterion':['gini', 'entropy'],
    'max_depth':[2, 5, 10, 20, 50]
}, cv=5)
clf.fit(training_X, training_y)
param_tree_results = pd.DataFrame(clf.cv_results_)
trimmed_tree_results = param_tree_results[['param_criterion', 'param_max_depth', 'mean_test_score', 'std_test_score']]


In [69]:
trimmed_tree_results.sort_values('mean_test_score', ascending=False)

,param_criterion,param_max_depth,mean_test_score,std_test_score
8,entropy,20,0.952453,0.012075
9,entropy,50,0.948679,0.007395
7,entropy,10,0.944151,0.010512
2,gini,10,0.939623,0.015467
3,gini,20,0.934340,0.014828
4,gini,50,0.934340,0.010832
1,gini,5,0.896604,0.019767
6,entropy,5,0.890566,0.025705
0,gini,2,0.843774,0.019911
5,entropy,2,0.831698,0.023699


---
### Random Forest

In [74]:
clf = GridSearchCV(RandomForestClassifier(), {
    'criterion':['gini', 'entropy'],
    'max_depth':[2, 5, 10, 20, 50],
    'n_estimators':[10, 50, 100, 200, 500]
}, cv=5)
clf.fit(training_X, training_y)
param_forest_results = pd.DataFrame(clf.cv_results_)
trimmed_forest_results = param_forest_results[['param_n_estimators', 'param_criterion', 'param_max_depth', 'mean_test_score', 'std_test_score']]

In [75]:
trimmed_forest_results.sort_values('mean_test_score', ascending=False)

,param_n_estimators,param_criterion,param_max_depth,mean_test_score,std_test_score
48,200,entropy,50,0.973585,0.005337
42,100,entropy,20,0.973585,0.006750
24,500,gini,50,0.972075,0.006579
44,500,entropy,20,0.972075,0.005119
14,500,gini,10,0.971321,0.005648
23,200,gini,50,0.971321,0.005648
47,100,entropy,50,0.970566,0.004401
49,500,entropy,50,0.969811,0.007916
43,200,entropy,20,0.969811,0.004134
41,50,entropy,20,0.969811,0.007916


---
### K-Nearest Neighbor - H/K

In [71]:
clf = GridSearchCV(KNeighborsClassifier(), {
    'algorithm':['ball_tree', 'brute', 'kd_tree'],
    'n_neighbors':[1, 3, 5, 10, 20],
    'weights': ['uniform', 'distance']
}, cv=5)
clf.fit(training_X, training_y)
param_knn_results = pd.DataFrame(clf.cv_results_)
trimmed_knn_results = param_knn_results[['param_n_neighbors', 'param_algorithm', 'param_weights', 'mean_test_score', 'std_test_score']]

In [72]:
trimmed_knn_results.sort_values('mean_test_score', ascending=False)

,param_n_neighbors,param_algorithm,param_weights,mean_test_score,std_test_score
25,5,kd_tree,distance,0.955472,0.007317
5,5,ball_tree,distance,0.954717,0.008268
15,5,brute,distance,0.953208,0.007395
21,1,kd_tree,distance,0.953208,0.006131
20,1,kd_tree,uniform,0.953208,0.006131
10,1,brute,uniform,0.953208,0.006579
11,1,brute,distance,0.953208,0.006579
24,5,kd_tree,uniform,0.951698,0.009057
23,3,kd_tree,distance,0.951698,0.010237
1,1,ball_tree,distance,0.951698,0.007317
